In [0]:
from google.colab.auth import authenticate_user
authenticate_user()

In [0]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

In [7]:
!gsutil cp gs://ekaba-assets/datasets/QA/BioASQ/BioASQ-train-factoid-4b.json .
!gsutil cp gs://ekaba-assets/datasets/QA/BioASQ/BioASQ-train-factoid-5b.json .
!gsutil cp gs://ekaba-assets/datasets/QA/BioASQ/BioASQ-train-factoid-6b.json .
# !gsutil cp gs://ekaba-assets/datasets/QA/BioASQ/trainining7b.json .

Copying gs://ekaba-assets/datasets/QA/BioASQ/trainining7b.json...
- [1 files][ 24.6 MiB/ 24.6 MiB]                                                
Operation completed over 1 objects/24.6 MiB.                                     


In [0]:
df1 = pd.read_json('BioASQ-train-factoid-4b.json')
df2 = pd.read_json('BioASQ-train-factoid-5b.json')
df3 = pd.read_json('BioASQ-train-factoid-6b.json')
# df4 = pd.read_json('trainining7b.json')

In [0]:
l1 = len(df1['data'].iloc[0]['paragraphs'])
l2 = len(df2['data'].iloc[0]['paragraphs'])
l3 = len(df3['data'].iloc[0]['paragraphs'])
# l4 = len(df4['data'].iloc[0]['paragraphs'])

In [0]:
def get_df(l,df) :

    question = []
    q_id = []
    ans = []

    for i in tqdm(range(0,l)):

        para = df['data'].iloc[0]['paragraphs'][i]
        qas = para['qas'][0]

        question.append(qas['question'])
        q_id.append(qas['id'])
        ans.append(para['context'])
        
    
    
    #Dataframe
    df_qa = pd.DataFrame()
    
    
    #dataframe
    df_qa['id'] = q_id
    df_qa['question'] = question
    df_qa['answer'] = ans

    return df_qa



In [15]:
df_qa1 = get_df(l1,df1)
df_qa2 = get_df(l2,df2)
df_qa3 = get_df(l3,df3)
# df_qa4 = get_df(l4,df4)

100%|██████████| 4772/4772 [00:00<00:00, 73945.85it/s]


In [0]:
df_qa = pd.concat([df_qa1,df_qa2,df_qa3],sort=False)

In [0]:
df_qa = df_qa.drop(['id'],axis=1)

In [18]:
df_qa['answer'].nunique()

2582

In [0]:
df_qa_un = df_qa.drop_duplicates(keep='first')

In [20]:
df_qa_un.head()

,question,answer
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h..."
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...


In [22]:
df_qa_un.iloc[0]['question']

'What is the inheritance pattern of Li–Fraumeni syndrome?'

In [23]:
df_qa_un.iloc[0]['answer']

'Balanced t(11;15)(q23;q15) in a TP53+/+ breast cancer patient from a Li-Fraumeni syndrome family. Li-Fraumeni Syndrome (LFS) is characterized by early-onset carcinogenesis involving multiple tumor types and shows autosomal dominant inheritance. Approximately 70% of LFS cases are due to germline mutations in the TP53 gene on chromosome 17p13.1. Mutations have also been found in the CHEK2 gene on chromosome 22q11, and others have been mapped to chromosome 11q23. While characterizing an LFS family with a documented defect in TP53, we found one family member who developed bilateral breast cancer at age 37 yet was homozygous for wild-type TP53. Her mother also developed early-onset primary bilateral breast cancer, and a sister had unilateral breast cancer and a soft tissue sarcoma. Cytogenetic analysis using fluorescence in situ hybridization of a primary skin fibroblast cell line revealed that the patient had a novel balanced reciprocal translocation between the long arms of chromosomes 1

In [0]:
def text_preprocessing(text):
    text= text.lower()
    text= re.sub(r'[^a-z]',' ',text)
    #text= " ".join([s for s in text.split() if len(s)>2])
    #text= " ".join([x for x in text.split() if x not in stopwords.words('english')])
    #text= " ".join([inflection.singularize(x) for x in text.split()])
    text= text.strip()
    return(text)

In [0]:
tqdm.pandas()

In [26]:
df_qa_un['question'] = df_qa_un['question'].progress_apply(lambda x: text_preprocessing(x))
df_qa_un['answer'] =   df_qa_un['answer'].progress_apply(lambda x: text_preprocessing(x))

100%|██████████| 2607/2607 [00:00<00:00, 123808.86it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
100%|██████████| 2607/2607 [00:00<00:00, 15471.51it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
df_qa_un.shape

(2607, 2)

In [0]:
unique_ques = list(df_qa_un['question'].unique())
unique_ans =  list(df_qa_un['answer'].unique())

In [0]:
dict_ques = {}

for i,qs in enumerate(unique_ques):
    dict_ques[qs] = i    

In [0]:
dict_ans = {}

for i,ans in enumerate(unique_ans):
    dict_ans[ans] = i    

In [31]:
qid = []
pid = []


for i in tqdm(range(0,len(df_qa_un))):
    
    ques = df_qa_un['question'].iloc[i]
    ans = df_qa_un['answer'].iloc[i]
    
    qid.append(dict_ques[ques])
    pid.append(dict_ans[ans])

100%|██████████| 2607/2607 [00:00<00:00, 33683.43it/s]


In [32]:
df_qa_un['qid'] = qid
df_qa_un['pid'] = pid

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
df_qa_un.head()

,question,answer,qid,pid
0,what is the inheritance pattern of li fraumeni...,balanced t q q in a tp breast...,0,0
1,what is the inheritance pattern of li fraumeni...,genetic modeling of li fraumeni syndrome in ze...,0,1
2,which type of lung cancer is afatinib used for,clinical perspective of afatinib in non small ...,1,2
3,which hormone abnormalities are characteristic...,doca sensitive pendrin expression in kidney h...,2,3
4,which hormone abnormalities are characteristic...,clinical and molecular characteristics of pend...,2,4


In [34]:
df_qa_un[df_qa_un['answer'].str.contains('fraumeni')]

,question,answer,qid,pid
0,what is the inheritance pattern of li fraumeni...,balanced t q q in a tp breast...,0,0
1,what is the inheritance pattern of li fraumeni...,genetic modeling of li fraumeni syndrome in ze...,0,1
1757,which tumor suppressor is referred to as the ...,p a guide to apoptosis approximately o...,89,564
2153,what is the usual her status in breast cance...,breast cancer phenotype in women with tp ger...,124,741
2156,what is the usual her status in breast cance...,early onset her positive breast cancer is ass...,124,742
2159,what is the usual her status in breast cance...,a novel her positive breast cancer phenotype ...,124,743
2739,which is the gene mutated in type neurofibro...,inducible and transmissible genetic events and...,165,1135


In [35]:
df_qa_un[df_qa_un['answer'].str.contains('status')]

,question,answer,qid,pid
2,which type of lung cancer is afatinib used for,clinical perspective of afatinib in non small ...,1,2
32,which hormone abnormalities are characteristic...,analysis of the slc a gene in patients with ...,2,16
66,what is the name for anorexia in gymnasts,salivary adiponectin levels are associated wit...,6,33
120,which is the main calcium binding protein of t...,control of muscle ryanodine receptor calcium r...,9,59
136,what is the target of the drug olaparib,xrcc promotes colorectal cancer cell growth ...,11,65
...,...,...,...,...
4500,what condition is usually represented by the a...,unifying the definitions of sudden unexpected ...,407,2440
4513,symptoms of which disorder are evaluated with ...,the sf offers a strong measure of mental he...,410,2450
4688,what is the indication for valbenazine,valbenazine granted breakthrough drug status f...,431,2541
4692,what organism causes tularemia,the status of tularemia in europe in a one hea...,432,2545


In [0]:
#df_qa1['qid'].value_counts()

In [0]:
#df_qa1['pid'].value_counts()

We need to have more answers per question to have a more robust MRR metric. We will add more data synthetically to enhance the quality of the metric